<a href="https://colab.research.google.com/github/ishwarvenugopal/CE888_Data_Science_and_Decision_Making/blob/master/Lab_5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [40]:
df = pd.read_csv('jester-data-1.csv',header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [41]:
df=df.drop(df.columns[0],1)
print(df)

         1      2      3      4     5    ...    96     97     98     99     100
0      -7.82   8.79  -9.66  -8.16 -7.52  ...  99.00  -5.63  99.00  99.00  99.00
1       4.08  -0.29   6.36   4.37 -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
2      99.00  99.00  99.00  99.00  9.03  ...  99.00  99.00  99.00  99.00  99.00
3      99.00   8.35  99.00  99.00  1.80  ...  99.00  99.00  99.00  99.00  99.00
4       8.50   4.61  -4.17  -5.39  1.36  ...   1.55   3.11   6.55   1.80   1.60
...      ...    ...    ...    ...   ...  ...    ...    ...    ...    ...    ...
24978   0.44   7.43   9.08   2.33  3.20  ...   9.03   6.55   8.69   8.79   7.43
24979   9.13  -8.16   8.59   9.08  0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24980  99.00  99.00  99.00  99.00 -7.77  ...  99.00  99.00  99.00  99.00  99.00
24981  99.00  99.00  99.00  99.00 -9.71  ...  99.00  99.00  99.00  99.00  99.00
24982   2.43   2.67  -3.98   4.27 -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 100 columns]


In [42]:
arr = df.values
print(arr.shape)

(24983, 100)


In [43]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [46]:
arr[idx[0][0], idx[1][0]]

-9.37

In [47]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 10

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10) == 0:
        print(mse)

In [50]:
sgd()

25.632545841671114
24.924139189078133
24.866470147395876
24.812103261298667
24.75893376931546
24.705811908116633
24.651598660129363
24.59497636356289
24.534226783092702
24.466947570696643
24.389674717534042
24.297382190003
24.182852995531388
24.03598355229435
23.843239157889972
23.58777252543361
23.25113600498976
22.817793705947345
22.282997676702724
21.661995669999527
20.99449385688006
20.337289995111288
19.74536378002988
19.25299560293226
18.867574771589883
18.576892225347997
18.360827170376787
18.199692421441224
18.077668883976663
17.983158726528416
17.907985108790726
17.84644565544678
17.794549035127208
17.749471065942092
17.709184280228982
17.67220884235122
17.637445019009206
17.604059964527057
17.571410797926813
17.53899211543733
17.506400105403134
17.473308049321737
17.439449699659317
17.404608141401226
17.368608474650596
17.33131313298395
17.29261896476627
17.252455411745533
17.210783262003414
17.16759356125934


In [51]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-3.425, -4.235, -4.806, ..., -1.892, -4.929, -2.289],
       [ 3.03 ,  2.513,  3.123, ...,  1.536,  3.352,  2.434],
       [ 7.125,  6.995,  7.665, ...,  6.387,  6.726,  7.786],
       ...,
       [-1.16 , -0.867, -1.818, ...,  0.033, -2.497, -0.394],
       [-1.639, -1.799, -2.797, ..., -1.016, -1.541, -0.969],
       [ 2.885,  2.823,  2.858, ...,  4.452,  2.121,  4.358]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [55]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -3.4247056798704536)","(8.79, -4.234708045194466)","(-9.66, -4.806186448531996)","(-8.16, -7.669885160792961)","(-7.52, -2.073047379256809)","(-8.5, -3.0197209660992232)","(99.0, -2.8872787026449593)","(4.17, -3.6158652340067094)","(-8.98, -8.51180536497136)","(-4.76, -4.30545698212153)","(-8.5, -2.166452309170533)","(-6.75, -2.3182186446400643)","(-7.18, -5.720980300555779)","(8.45, -2.9501238787974056)","(-7.18, -5.113094752385717)","(-7.52, -8.188164299110204)","(-7.43, -3.629395797064132)","(-9.81, -4.7663359609510145)","(-9.85, -3.1897810289579884)","(-9.85, -4.8717820684222)","(-9.37, -0.6935190599295737)","(1.5, -5.2140445227845955)","(99.0, -5.8388004372000974)","(-9.81, -8.991825359804809)","(-8.5, -4.833538087889739)","(1.12, -2.6344488772367316)","(7.82, 1.7612439465018157)","(2.86, -2.8596918616012106)","(9.13, 1.651471479925005)","(-7.43, -7.354595795299863)","(2.14, -0.41723933839398575)","(-4.08, 1.5026609012303205)","(99.0, -7.442770939469019)","(7.82, -3.450437548366836)","(5.05, 1.280053773091425)","(4.95, 1.5522863249048513)","(-9.17, -7.596514809151933)","(-8.4, -4.391234801321946)","(-8.4, -2.9089803543424337)","(-8.4, -3.5446407629756576)",...,"(8.59, -0.7069352477667643)","(99.0, -0.02319198408339914)","(-6.84, -5.130715989736906)","(-9.03, -7.096784110954341)","(2.82, -0.12881044687561458)","(99.0, -0.3089835889534244)","(-9.08, -8.642289485425469)","(99.0, -0.21830413570070906)","(5.68, 0.12865325103756095)","(-4.81, -4.590594443285062)","(99.0, -4.947646834830312)","(99.0, 0.33694926869842057)","(99.0, -2.678068534035496)","(99.0, -7.023834823861724)","(99.0, -5.060406756440247)","(99.0, 0.058877719686740625)","(99.0, -3.114599459251947)","(-9.42, -1.8238724744628503)","(99.0, -4.270884689727091)","(99.0, -2.3151124907073064)","(99.0, -1.4644442602425927)","(-7.72, -3.7780071670459057)","(99.0, -0.4980365627263991)","(99.0, -4.001263456045091)","(99.0, -3.714942908131693)","(99.0, -4.8854829004496905)","(99.0, -1.246359164282403)","(99.0, -1.6836252964683713)","(99.0, 2.326499711247227)","(99.0, -3.7367658947440994)","(99.0, -1.5554608076113265)","(99.0, -2.4792754101858425)","(99.0, 0.057217900096210014)","(99.0, -4.026677174739992)","(99.0, -3.4255930241257104)","(99.0, -3.0002391598232605)","(-5.63, -1.9371436401776654)","(99.0, -1.8921870282279325)","(99.0, -4.92870323725428)","(99.0, -2.2886447584869045)"
1,"(4.08, 3.030407160667882)","(-0.29, 2.5131855904296314)","(99.0, 3.1225226157912926)","(4.37, 2.3585991015029655)","(-2.38, 1.7854046734625748)","(-9.66, 3.8826657427791766)","(-0.73, 1.0458362634129772)","(-5.34, 1.2496105743830053)","(8.88, 4.403225784720627)","(9.22, 3.890972306882566)","(6.75, 3.6388923748029747)","(8.64, 3.2327350065245932)","(4.42, 0.9583686562687411)","(7.43, 3.528403813869612)","(4.56, 0.9581369704123508)","(-0.97, 1.3561266144435287)","(4.66, 0.5540260256690333)","(99.0, 2.0309861261541684)","(3.3, 2.150257795105379)","(-1.21, 1.7197176078841354)","(0.87, 3.0483610035020945)","(8.64, 4.010730888111507)","(99.0, 3.8258825432504895)","(9.17, 3.3622009708795653)","(0.05, 3.0199982523497657)","(7.57, 2.892036802847481)","(4.71, 2.907717615950775)","(99.0, 3.97791506843126)","(-0.39, 2.3187804497609887)","(6.99, 3.9982505549875325)","(6.5, 3.1830186685470157)","(-0.92, 3.126300117737592)","(7.14, 2.7469535139519867)","(9.03, 3.211380072159967)","(-1.8, 2.8628160630402704)","(99.0, 2.9755800566491692)","(7.09, 2.607553986125717)","(3.4, 4.292563231994186)","(-0.87, 3.085549768186105)","(7.91, 3.349329819840058)",...,"(99.0, 3.52094479058308)","(-3.35, 4.0543529591896865)","(-9.03, 3.6030671922156032)","(4.47, 3.402896834482027)","(4.08, 3.0795795869304645)","(-3.83, 3.430820274575623)","(8.74, 4.208472951093977)","(1.12, 3.605450